# Importamos las librerias

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import backend as k
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import softmax


#Limpiamos todas las sesiones para tener más rendimiento


In [ ]:
k.clear_session()

#Para conectarnos con google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Descomprimiendo los archivos ZIP de nuestro set de datos

In [ ]:
!unzip /content/drive/MyDrive/Felinos.zip -d /content/

# Definimos la ruta del set de datos tanto de entrenamiento como de validación

In [ ]:
#Ruta de la carpeta con las imágenes de entrenamiento
data_entrenamiento = '/content/Felinos/train'

#Ruta de la carpeta con las imágenes de validación
data_validacion = '/content/Felinos/val'

#Contando el número de imágenes del set de datos

In [ ]:
#Variables para guardar el numero de clases para entrenamiento y validación
num_clases_entrenamiento = 0
num_clases_validacion = 0

#Variables para guardar el numero de imagenes para entrenamiento y validación
num_imagenes_entrenamiento = 0
num_imagenes_validacion = 0

#Recorremos cada subcarpeta de la ruta de entrenamiento
for root, dirs, files in os.walk(data_entrenamiento):
    for dir in dirs:
        carpeta_ruta = os.path.join(root, dir)
        num_imagenes = len(os.listdir(carpeta_ruta)) #Obtenemos el total de imagenes para x clase
        num_clases_entrenamiento += 1 #Aumentamos el número de clases
        num_imagenes_entrenamiento = num_imagenes_entrenamiento + num_imagenes #Aumentamos el total de imagenes por cada clase
        print(f'El número de imágenes en la carpeta {carpeta_ruta} es: {num_imagenes}')

print("\n\n")

#Recorremos cada subcarpeta de la ruta de validación
for root, dirs, files in os.walk(data_validacion):
    for dir in dirs:
        carpeta_ruta = os.path.join(root, dir)
        num_imagenes = len(os.listdir(carpeta_ruta))  #Obtenemos el total de imagenes para x clase
        num_clases_validacion += 1 #Aumentamos el número de clases
        num_imagenes_validacion = num_imagenes_validacion + num_imagenes #Aumentamos el total de imagenes por cada clase
        print(f'El número de imágenes en la carpeta {carpeta_ruta} es: {num_imagenes}')

#Mostramos el numero de clases y el numero de imagenes de entrenamiento
print(f'\n\nEl número de clases en la carpeta {data_entrenamiento} es: {num_clases_entrenamiento}')
print(f'El número de imagenes de entrenamiento es: {num_imagenes_entrenamiento}')

#Mostramos el numero de clases y el numero de imagenes de validación
print(f'\n\nEl número de clases en la carpeta {data_validacion} es: {num_clases_validacion}')
print(f'El número de imagenes de validación es: {num_imagenes_validacion}')

# Definimos parámetros e hiperparámetros para la CNN

In [ ]:
altura, longitud = 224, 224 #Tamaño de las imágenes que se utilizarán
batch_size = 32 #Tamaño del lote para el entrenamiento
epocas = 100 #Cantidad de épocas para entrenar
pasos = num_imagenes_entrenamiento//batch_size #Número de pasos de entrenamiento por época
validation_steps = num_imagenes_validacion//batch_size #Número de pasos de validación por épocas
filtros_Conv1 = 32 #Número de filtros para la primera capa convolucional
filtros_Conv2 = 64 #Número de filtros para la segunda capa convolucional
filtros_Conv3 = 128 #Número de filtros para la tercera capa convolucional
filtros_Conv4 = 256 #Número de filtros para la cuarta capa convolucional
filtros_Conv5 = 512 #Número de filtros para la quinta capa convolucional
tamanio_filtro1 = (3, 3) #Tamaño del filtro para la primera capa convolucional
tamanio_filtro2 = (3, 3) #Tamaño del filtro para la segunda capa convolucional
tamanio_filtro3 = (3, 3) #Tamaño del filtro para la tercera capa convolucional
tamanio_filtro4 = (3, 3) #Tamaño del filtro para la cuarta capa convolucional
tamanio_filtro5 = (3, 3) #Tamaño del filtro para la quinta capa convolucional
tamanio_pool = (2, 2) #Tamaño de pool de agrupación
clases = 4 #Número de clases (leon, gato, tigre y pantera)
lr = 0.0004 #Tasa de aprendizaje

# Definimos el ImageDataGenerator para la generación de imágenes y ajuste

In [ ]:
#Generación de datos de imágenes y ajuste para entrenamiento
entrenamiento_datagen = ImageDataGenerator(
    rescale=1./255,               #Reescala los valores de píxeles para que estén en el rango [0, 1]
    rotation_range=10,            #Rango de ángulos en grados para rotar aleatoriamente las imágenes
    vertical_flip=True,           #Voltea verticalmente las imágenes de forma aleatoria
    width_shift_range=0.3,        #Desplazamiento aleatorio horizontal en fracción de la anchura total de la imagen
    height_shift_range=0.1,       #Desplazamiento aleatorio vertical en fracción de la altura total de la imagen
    channel_shift_range=0.2,      #Desplazamiento aleatorio de los canales de color
    shear_range=0.2,              #Rango de ángulos en grados para realizar transformaciones de cizallamiento
    zoom_range=0.2,               #Rango de zoom aleatorio para las imágenes
    horizontal_flip=True,         #Voltea horizontalmente las imágenes de forma aleatoria
    brightness_range=(0.8, 1.2),  #Rango de brillo aleatorio para las imágenes
    fill_mode='nearest')          #Modo de relleno para los píxeles cuando se aplican transformaciones de tamaño o posición

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(
    data_entrenamiento,
    target_size=(longitud, altura),
    batch_size=batch_size,
    class_mode='categorical')

#Generación de datos de imágenes y ajuste para validación
validacion_datagen = ImageDataGenerator(rescale=1./255) #Solo se reescalan los valores de píxeles para la validación

validacion_generador = validacion_datagen.flow_from_directory(
    data_validacion,
    target_size=(longitud, altura),
    batch_size=batch_size,
    class_mode='categorical')

#Obtenemos los nombres de las clases y las guardamos en un archivo json para usarlo en un archivo aparte para la predicción

In [ ]:
#Obtenemos los nombres de las clases
nombre_clases = entrenamiento_generador.class_indices

#Guardamos los nombres de las clases en un archivo JSON
with open('clases_Felino.json', 'w') as f:
    json.dump(nombre_clases, f)


# Definimos la arquitectura de la CNN

In [ ]:
#Creación del modelo
modelo = Sequential()

#Primera apa de convolución y pooling
modelo.add(Conv2D(filtros_Conv1, tamanio_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
modelo.add(MaxPooling2D(pool_size=tamanio_pool))

#Segunda capa de convolución y pooling
modelo.add(Conv2D(filtros_Conv2, tamanio_filtro2, padding="same", activation='relu'))
modelo.add(MaxPooling2D(pool_size=tamanio_pool))

#Tercera capa de convolución y pooling
modelo.add(Conv2D(filtros_Conv3, tamanio_filtro3, padding="same", activation='relu'))
modelo.add(MaxPooling2D(pool_size=tamanio_pool))

#Cuarta capa de convolución y pooling
modelo.add(Conv2D(filtros_Conv4, tamanio_filtro4, padding="same", activation='relu'))
modelo.add(MaxPooling2D(pool_size=tamanio_pool))

#Quinta capa de convolución y pooling
modelo.add(Conv2D(filtros_Conv5, tamanio_filtro5, padding="same", activation='relu'))
modelo.add(MaxPooling2D(pool_size=tamanio_pool))

#Capa completamente conectada
modelo.add(Flatten())
modelo.add(Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
modelo.add(Dropout(0.5))
modelo.add(Dense(clases, activation='softmax'))

# Compilamos el modelo

In [ ]:
modelo.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(learning_rate=lr),
              metrics=['accuracy'])

# Utilizamos el callback "checkpoint" y el callback "earlystopping" para guardar el modelo con mejor rendimiento y con una paciencia de 10

In [ ]:
#Configuramos el callback chekpoint (content/Mejor_Modelo/modelo_felinos.h5)
checkpoint = ModelCheckpoint(filepath='/content/drive/MyDrive/modelo_felinos.h5', #Para que se guarde el modelo directamente a nuestro drive (y para guardarlo en el entorno de forma temporal usar: /content/Mejor_Modelo/modelo_felinos.h5
                             monitor='val_accuracy', 
                             mode='max', 
                             save_best_only=True, 
                             verbose=1)

#Configuramos el callback earlystopping con una paciencia de 10
earlystopping = EarlyStopping(monitor='val_accuracy',
                              patience=10, 
                              verbose=1)

#Unimos ambos callbacks
callbacks = [checkpoint, earlystopping]

# Entrenamos el modelo

In [ ]:
historial_entrenamiento = modelo.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps,
    callbacks=callbacks)

#Cargamos el mejor modelo obtenido

In [ ]:
modelo_cargado = load_model('/content/drive/MyDrive/modelo_felinos.h5') #Cargar el modelo guardado desde nuestro drive
#modelo_cargado = load_model('/content/Mejor_Modelo/modelo_felinos.h5') #Cargar el modelo desde la carpeta Mejor_Modelo

#Obtenemos el accuracy y loss del historial de entrenamiento

In [ ]:
obtener_metricas = historial_entrenamiento.history

#Mostramos las métricas del historial
print(obtener_metricas.keys())

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])


#Graficamos las metricas accuracy y loss de los datos de entrenamiento

In [ ]:
#Trazamos la curva de pérdida obtenida durante el entrenamiento
plt.plot(obtener_metricas['loss'], 'ro--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de pérdida - Entrenamiento')
plt.show()

#Trazamos la curva de precisión obtenida durante el entrenamiento
plt.plot(obtener_metricas['accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Curva de precisión - Entrenamiento')
plt.show()

#Trazamos la curva de pérdida y la curva de precisión para entrenamiento
plt.plot(obtener_metricas['loss'], 'ro--')
plt.plot(obtener_metricas['accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida/Precisión')
plt.title('Curva de precisión/pérdida - Entrenamiento')
plt.show()

#Graficamos las metricas accuracy y loss de los datos de validación

In [ ]:
#Trazamos la curva de pérdida de validación obtenida durante el entrenamiento
plt.plot(obtener_metricas['val_loss'], 'ro--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de pérdida - Validación')
plt.show()

#Trazamos la curva de precisión de validación obtenida durante el entrenamiento
plt.plot(obtener_metricas['val_accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Curva de precisión - Validación')
plt.show()

#Trazamos la curva de pérdida y la curva de precisión para validación
plt.plot(obtener_metricas['val_loss'], 'ro--')
plt.plot(obtener_metricas['val_accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida/Precisión')
plt.title('Curva de precisión/pérdida - Validación')
plt.show()

#Realizamos una predicción de alguna imágen del set de datos de validación

In [ ]:
#Cargamos una imagen del set de validación
ruta_imagen = '/content/Felinos/val/pantera/pantera_val_77.jpg'
img = load_img(ruta_imagen, target_size=(altura, longitud))

#Convertimos la imagen a un array de NumPy
img_array = img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

#Hacemos la predicción con el modelo
prediccion = modelo_cargado.predict(img_array)

#Obtener las probabilidades de pertenencia a cada clase y aplicar la función softmax
probabilidades = softmax(prediccion)[0]

#Obtener los nombres de las clases
nombre_clases = entrenamiento_generador.class_indices.keys()

#Imprimimos las clases de felinos
print(nombre_clases)

#Imprimir los porcentajes de pertenencia a cada clase
for i, name in enumerate(nombre_clases):
    print(f"{name}: {probabilidades[i]*100:.2f}%")

#Opcional
En caso de no quedar satisfecho con el entrenamiento y se quiera volver a entrenar el mejor modelo del entrenamiento anterior y así no iniciar de cero.

In [ ]:
historial_modelo_cargado = modelo_cargado.fit(
    entrenamiento_generador,
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,
    validation_steps=validation_steps,
    callbacks=callbacks)

#Opcional
Obtenemos el accuracy y loss del historial de entrenamiento

In [ ]:
obtener_metricas_modelo_nuevo = historial_modelo_cargado.history

#Mostramos las métricas del historial
print(obtener_metricas_modelo_nuevo.keys())

#Opcional
Graficamos las metricas accuracy y loss de los datos de entrenamiento

In [ ]:
#Trazamos la curva de pérdida obtenida durante el entrenamiento
plt.plot(obtener_metricas_modelo_nuevo['loss'], 'ro--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de pérdida - Entrenamiento')
plt.show()

#Trazamos la curva de precisión obtenida durante el entrenamiento
plt.plot(obtener_metricas_modelo_nuevo['accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Curva de precisión - Entrenamiento')
plt.show()

#Trazamos la curva de pérdida y la curva de precisión para entrenamiento
plt.plot(obtener_metricas_modelo_nuevo['loss'], 'ro--')
plt.plot(obtener_metricas_modelo_nuevo['accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida/Precisión')
plt.title('Curva de precisión/pérdida - Entrenamiento')
plt.show()

#Opcional
Graficamos las metricas accuracy y loss de los datos de validación

In [ ]:
#Trazamos la curva de pérdida de validación obtenida durante el entrenamiento
plt.plot(obtener_metricas_modelo_nuevo['val_loss'], 'ro--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.title('Curva de pérdida - Validación')
plt.show()

#Trazamos la curva de precisión de validación obtenida durante el entrenamiento
plt.plot(obtener_metricas_modelo_nuevo['val_accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Precisión')
plt.title('Curva de precisión - Validación')
plt.show()

#Trazamos la curva de pérdida y la curva de precisión para validación
plt.plot(obtener_metricas_modelo_nuevo['val_loss'], 'ro--')
plt.plot(obtener_metricas_modelo_nuevo['val_accuracy'], 'go--')
plt.xlabel('Épocas')
plt.ylabel('Pérdida/Precisión')
plt.title('Curva de precisión/pérdida - Validación')
plt.show()